# Packages

In [1]:
! pip install -q langchain_community tiktoken langchain-openai langchainhub chromadb langchain langchain-core langchain_google_genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.2/396.2 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 8.4 MB/s eta 0:00:0

# Langchain and Groq api

In [2]:
import os
from google.colab import userdata

# Retrieve the API key
langchain_api = userdata.get('langchain_api')
groq_api = userdata.get('groq_api')

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = "langchain_api"
os.environ['GROQ_API_KEY'] = 'groq_api'

In [24]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

#### INDEXING ####

# Load Documents
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

loader = WebBaseLoader("https://lilianweng.github.io/posts/2024-02-05-human-data-quality/")
docs.extend(loader.load())

In [ ]:
!pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 3.8 MB/s eta 0:00:00


In [ ]:
from langchain_groq import ChatGroq
from langchain.retrievers.multi_query import MultiQueryRetriever

# creating multi queries
llm = ChatGroq(model_name='llama-3.1-70b-versatile')

In [26]:
import uuid #uuid: This module is used to generate unique identifiers for documents.

from langchain_core.documents import Document #This class from Langchain represents a document with content and metadata.
from langchain_core.output_parsers import StrOutputParser # This is used to parse the output of the language model into a string format.
from langchain_core.prompts import ChatPromptTemplate #This class from Langchain helps create structured prompts for chat-based models.

chain = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template("Summarize the following document:\n\n{doc}")
    | llm
    | StrOutputParser()
)
# chain.batch: This method applies the chain pipeline to the list of documents in batches, allowing for parallel processing.
summaries = chain.batch(docs, {"max_concurrency": 5})

In [27]:
summaries

['This article, "LLM Powered Autonomous Agents," discusses the concept of building autonomous agents using large language models (LLMs) as their core controller. The author, Lilian Weng, explores the potential of LLMs as a powerful general problem solver, going beyond generating text and solving simple tasks.\n\n**Agent System Overview**\n\nThe article describes a LLM-powered autonomous agent system, consisting of three key components:\n\n1. **Planning**: The agent breaks down complex tasks into smaller, manageable subgoals, using techniques such as task decomposition and self-reflection.\n2. **Memory**: The agent uses two types of memory: short-term memory (in-context learning) and long-term memory (external vector store).\n3. **Tool use**: The agent learns to call external APIs for extra information, including current information, code execution capability, and access to proprietary information sources.\n\n**Component One: Planning**\n\nThe article discusses various techniques for ta

In [31]:
!pip install sentence-transformers

In [28]:
from langchain.storage import InMemoryByteStore #InMemoryByteStore: A non-persistent storage solution that holds data in memory using a Python dictionary.
from langchain_community.vectorstores import Chroma
from langchain.retrievers.multi_vector import MultiVectorRetriever
import chromadb.utils.embedding_functions as embedding_functions
from langchain.embeddings import HuggingFaceEmbeddings

# Initialize the Hugging Face Sentence Transformer model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# The vectorstore to use to index the child chunks
vectorstore = Chroma(collection_name="summaries",
                     embedding_function=embeddings)

# The storage layer for the parent documents

"""An InMemoryByteStore is instantiated to store metadata or additional
information about the documents that are not directly related to the embeddings.
The id_key is defined to specify the key used for document identifiers.
"""

store = InMemoryByteStore() #InMemoryByteStore: A non-persistent storage solution that holds data in memory using a Python dictionary.
id_key = "doc_id"

# MultiVectorRetriever: A retriever that can handle multiple vector stores
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)

doc_ids = [str(uuid.uuid4()) for _ in docs]

# Docs linked to summaries
summary_docs = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(summaries)
]

# Add
retriever.vectorstore.add_documents(summary_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))

<ipython-input-28-1a5d2346ab9c>:12: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vectorstore = Chroma(collection_name="summaries",


In [29]:
query = "Memory in agents"
sub_docs = vectorstore.similarity_search(query,k=1)
sub_docs[0]

Document(metadata={'doc_id': '68b8af44-76a0-4e3f-9a77-6588d6d92de9'}, page_content='This article, "LLM Powered Autonomous Agents," discusses the concept of building autonomous agents using large language models (LLMs) as their core controller. The author, Lilian Weng, explores the potential of LLMs as a powerful general problem solver, going beyond generating text and solving simple tasks.\n\n**Agent System Overview**\n\nThe article describes a LLM-powered autonomous agent system, consisting of three key components:\n\n1. **Planning**: The agent breaks down complex tasks into smaller, manageable subgoals, using techniques such as task decomposition and self-reflection.\n2. **Memory**: The agent uses two types of memory: short-term memory (in-context learning) and long-term memory (external vector store).\n3. **Tool use**: The agent learns to call external APIs for extra information, including current information, code execution capability, and access to proprietary information sources.

In [30]:
retrieved_docs = retriever.get_relevant_documents(query,n_results=1)
retrieved_docs[0].page_content[0:500]

"\n\n\n\n\n\nLLM Powered Autonomous Agents | Lil'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLil'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPosts\n\n\n\n\nArchive\n\n\n\n\nSearch\n\n\n\n\nTags\n\n\n\n\nFAQ\n\n\n\n\nemojisearch.app\n\n\n\n\n\n\n\n\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\n \n\n\nTable of Contents\n\n\n\nAgent System Overview\n\nComponent One: Planning\n\nTask Decomposition\n\nSelf-Reflection\n\n\nComponent Two: Memory\n\nTypes of Memory\n\nMaximum Inner Product Search (MIPS)\n\n"